# Memory

<div style="display: flex; justify-content: flex-start; gap: 10px;">
  <img src="./assets/LC_Memory_before.png" style="width:300px; border:1px solid #ccc; border-radius:6px;">
</div>

Persisting messages, or 'agent state' between invocations of the agent.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=****tM0A
LANGSMITH_API_KEY=****bc46
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [3]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

In [4]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [5]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [6]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-5",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

## Repeated Queries

In [7]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    context=RuntimeContext(db=db),
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_2S0enDthnwmpbrkxC91zOnxF)
 Call ID: call_2S0enDthnwmpbrkxC91zOnxF
  Args:
    query: SELECT name FROM sqlite_master WHERE type = 'table' AND (name LIKE '%invoice%' OR name LIKE '%invoices%' OR name LIKE '%customer%' OR name LIKE '%client%') ORDER BY name LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('Customer',), ('Invoice',), ('InvoiceLine',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_Z2RmX1J413Zbq5DoraIDv9Vr)
 Call ID: call_Z2RmX1J413Zbq5DoraIDv9Vr
  Args:
    query: SELECT i.InvoiceId, i.InvoiceDate, i.Total
FROM Customer c
JOIN Invoice i ON i.CustomerId = c.CustomerId
WHERE c.FirstName = 'Frank' A

In [8]:
question = "What were the titles?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================

Could you clarify which titles you mean? For example, titles of books, articles, movies, or a specific table/column in the database. Also, do you want more than 5 results?


## Add memory

In [9]:
from langgraph.checkpoint.memory import InMemorySaver

In [10]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    model="openai:gpt-5",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),
)

In [11]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_lGofl6niXaXss0BQy3cRR9KK)
 Call ID: call_lGofl6niXaXss0BQy3cRR9KK
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table' AND name LIKE '%invoice%' COLLATE NOCASE ORDER BY name LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('Invoice',), ('InvoiceLine',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_utJ20qvSbhqqJCDDfY560sMf)
 Call ID: call_utJ20qvSbhqqJCDDfY560sMf
  Args:
    query: SELECT sql FROM sqlite_master WHERE type='table' AND name='Invoice'
================================= Tool Message =================================
Name: execute_sql

[('CREATE TABLE [Invoice]\n(\n    [InvoiceId] 

In [12]:
question = "What were the titles?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_ADdee6YB0zjyxBqHn7E2azcW)
 Call ID: call_ADdee6YB0zjyxBqHn7E2azcW
  Args:
    query: SELECT t.Name AS TrackTitle
FROM InvoiceLine AS il
JOIN Track AS t ON t.TrackId = il.TrackId
WHERE il.InvoiceId = 374
ORDER BY il.InvoiceLineId ASC
LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('Holier Than Thou',), ('Through The Never',), ('My Friend Of Misery',), ('The Wait',), ('Blitzkrieg',)]
================================== Ai Message ==================================

Here are the track titles on that invoice:
- Holier Than Thou
- Through The Never
- My Friend Of Misery
- The Wait
- Blitzkrieg


## Try your own queries
Now that there is memory, check the agents recall!

In [14]:
question = "How many customers are there?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

How many customers are there?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_AGw2prVDuVa5OUwc70p6hsmy)
 Call ID: call_AGw2prVDuVa5OUwc70p6hsmy
  Args:
    query: SELECT COUNT(*) AS CustomerCount FROM Customer;
================================= Tool Message =================================
Name: execute_sql

[(59,)]
================================== Ai Message ==================================

There are 59 customers.


In [15]:
question = "What are their names?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What are their names?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_wqFSd97XWYujBxCftRUhh1Km)
 Call ID: call_wqFSd97XWYujBxCftRUhh1Km
  Args:
    query: SELECT c.FirstName, c.LastName
FROM Customer AS c
ORDER BY c.LastName, c.FirstName
LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('Roberto', 'Almeida'), ('Julia', 'Barnett'), ('Camille', 'Bernard'), ('Michelle', 'Brooks'), ('Robert', 'Brown')]
================================== Ai Message ==================================

Here are the first 5 customer names (alphabetical by last name):
- Roberto Almeida
- Julia Barnett
- Camille Bernard
- Michelle Brooks
- Robert Brown

Want me to list all 59 names?


In [16]:
question = "Yes"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

Yes
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_5RaQm35TfOPy0TnATfA8VqwV)
 Call ID: call_5RaQm35TfOPy0TnATfA8VqwV
  Args:
    query: SELECT c.FirstName, c.LastName
FROM Customer AS c
ORDER BY c.LastName, c.FirstName;
================================= Tool Message =================================
Name: execute_sql

[('Roberto', 'Almeida'), ('Julia', 'Barnett'), ('Camille', 'Bernard'), ('Michelle', 'Brooks'), ('Robert', 'Brown'), ('Kathy', 'Chase'), ('Richard', 'Cunningham'), ('Marc', 'Dubois'), ('João', 'Fernandes'), ('Edward', 'Francis'), ('Wyatt', 'Girard'), ('Luís', 'Gonçalves'), ('John', 'Gordon'), ('Tim', 'Goyer'), ('Patrick', 'Gray'), ('Astrid', 'Gruber'), ('Diego', 'Gutiérrez'), ('Bjørn', 'Hansen'), ('Frank', 'Harris'), ('Helena', 'Holý'), ('Phil', 'Hughes'), ('Terhi', 'Hämäläinen'), ('Joakim', 'Johansson'), ('Emma', 'Jones'), ('L